In [14]:
import sys
import os
import ujson
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from sklearn.linear_model import LinearRegression
from epivizfileserver.measurements import WebServerMeasurement
sys.path.insert(0,'C:/Users/Kyle/Documents/Python Scripts/Envs/research_epiviz/efeedcomputation/src/epivizfeedcompute')
from stats import BaseStats, TtestBlock, TtestExp, Correlation, CorrelationGeneSignal, OverlapBlock
import pickle


In [15]:
data = []
mid = "lung_cancer"
start=1
end=166151839
signal_data = pd.read_csv('timp2014_probelevel_beta_202001091200.csv')[["chr", "start", "end", "lung_normal", "lung_cancer"]]
signal_data = signal_data.where((signal_data.start >= start) & (signal_data.end <= end))
signal_data = signal_data.where((signal_data.chr != 'chrY'))
signal_data = signal_data.dropna()

In [16]:
signal_data

,chr,start,end,lung_normal,lung_cancer
0,chr1,15865.0,15865.0,0.9026,0.8800
1,chr1,18827.0,18827.0,0.6465,0.6145
2,chr1,29407.0,29407.0,0.0731,0.0985
3,chr1,29425.0,29425.0,0.1858,0.2073
4,chr1,29435.0,29435.0,0.0391,0.0353
...,...,...,...,...,...
485091,chrX,154842736.0,154842736.0,0.1320,0.2107
485092,chrX,154842841.0,154842841.0,0.3342,0.3300
485093,chrX,154842856.0,154842856.0,0.5510,0.4668
485094,chrX,154843228.0,154843228.0,0.6587,0.6618


In [17]:
# config_file = os.getcwd() + "/config.json"
# config_measurements = []
# with open(config_file, "r") as config_file:
#     data = ujson.loads(config_file.read())
#     # computations = data["computations"]
#     # info = data["dataSources"]
#     # pval_threshold = data["pval_threshold"]
#     if data["measurements"] is not None:
#         measurements = data["measurements"]
#         for m in measurements: 
#             tm = WebServerMeasurement(m['type'], m['id'], m['name'],  "http://54.157.53.251/api/", 
#                     m['datasourceId'], m['datasourceGroup'], m['annotation'], m['metadata'])
#             if tm.annotation is None:
#                 tm.annotation = {}
#             tm.annotation["datatype"] = m["datatype"]
#             config_measurements.append(tm)
# #ESR1 chr6: 150204511 - 157531913
# #ATOH7 chr10: 63661013 - 71027315
# chrom = "chr11"
# start=1
# end=166151839
# # filter for overlap measuremnets
# overlap_measurements = [m for m in config_measurements if m.mid in ["cpg_islands", "promoter"] ]
# overlap_measurements
# data = []

# for m in overlap_measurements:
#     data.append(m.get_data(chrom, start, end))
# print("done")

In [18]:
def split_n_buckets(dataframe, n):

    means = []
    dfs = np.array_split(dataframe, n,axis = 0) 
    for i in range(n):
        means.append(np.mean(dfs[i]))
    return means

In [19]:
dataF = pd.DataFrame() 
for i in range(1,23):
    query = 'http://54.157.53.251/api/?requestId=11&version=5&action=getValues&datasourceGroup=umd&datasource=gene_expression_barcode_subtype&measurement=lung___tumor,lung___normal&seqName=chr{}&start=1&end=166151839&genome=&_=1573233064747'.format(i)
    result = requests.get(query)
    # res = umsgpack.unpackb(result.content)
    res = result.json()

    data = res['data']

    if data['rows']['useOffset']:
        data['rows']['values']['start'] = np.cumsum(data['rows']['values']['start'])
        data['rows']['values']['end'] = np.cumsum(data['rows']['values']['end'])

    # convert json to dataframe
    records = {}

    for key in data['rows']['values'].keys():
        if key not in ["id", "strand", "metadata"]:
            records[key] = data['rows']['values'][key]

    for key in data['rows']['values']['metadata'].keys():
        records[key] = data['rows']['values']['metadata'][key]

    for key in data['values']['values'].keys():
        records[key] = data['values']['values'][key]
    temp = pd.DataFrame(records)
    dataF = pd.concat([dataF,temp])

dataF.head()


,end,start,chr,lung___normal,lung___tumor
0,894670.0,861118.0,chr1,0.0,0.025194
1,901095.0,895967.0,chr1,0.0,0.007752
2,901095.0,895967.0,chr1,0.0,0.003876
3,910488.0,901877.0,chr1,0.0,0.027132
4,917497.0,910579.0,chr1,0.0,0.007752


In [20]:
for i in ['X']:
    query = 'http://54.157.53.251/api/?requestId=11&version=5&action=getValues&datasourceGroup=umd&datasource=gene_expression_barcode_subtype&measurement=lung___tumor,lung___normal&seqName=chr{}&start=1&end=166151839&genome=&_=1573233064747'.format(i)
    result = requests.get(query)
    # res = umsgpack.unpackb(result.content)
    res = result.json()

    data = res['data']

    if data['rows']['useOffset']:
        data['rows']['values']['start'] = np.cumsum(data['rows']['values']['start'])
        data['rows']['values']['end'] = np.cumsum(data['rows']['values']['end'])

    # convert json to dataframe
    records = {}

    for key in data['rows']['values'].keys():
        if key not in ["id", "strand", "metadata"]:
            records[key] = data['rows']['values'][key]

    for key in data['rows']['values']['metadata'].keys():
        records[key] = data['rows']['values']['metadata'][key]

    for key in data['values']['values'].keys():
        records[key] = data['values']['values'][key]
    temp = pd.DataFrame(records)
    dataF = pd.concat([dataF,temp])


In [21]:
dataF.shape
# dataF['chr'].unique()

(28589, 5)

In [9]:
in_gene = []
methy_upstream = []
methy_downstream = []
upstream = 1000
downstream = 3000
for index, row in dataF.iterrows():
    in_gene_buckets = split_n_buckets(signal_data.where(((row.start <= signal_data.start) & (signal_data.start <= row.end)) | ((row.start <= signal_data.end) & (signal_data.end <= row.end))).fillna(0)[mid].tolist(), 20)
    methy_upstream_buckets = split_n_buckets(signal_data.where(((row.start - upstream<= signal_data.start) & (signal_data.start <= row.end)) | ((row.start - upstream <= signal_data.end) & (signal_data.end <= row.end))).fillna(0)[mid].tolist(), 10)
    methy_downstream_buckets = split_n_buckets(signal_data.where(((row.start <= signal_data.start) & (signal_data.start <= row.end + downstream)) | ((row.start <= signal_data.end) & (signal_data.end <= row.end + downstream))).fillna(0)[mid].tolist(), 10)
    in_gene.append(in_gene_buckets)
    methy_upstream.append(methy_upstream_buckets)
    methy_downstream.append(methy_downstream_buckets)


In [ ]:

in_gene = []
methy_upstream = []
methy_downstream = []
upstream = 1000
downstream = 3000
for index, row in dataF.iterrows():
    in_gene.append(signal_data.where(((row.start <= signal_data.start) & (signal_data.start <= row.end)) | ((row.start <= signal_data.end) & (signal_data.end <= row.end))).mean().fillna(0)[mid])
    methy_upstream.append(signal_data.where(((row.start - upstream<= signal_data.start) & (signal_data.start <= row.end)) | ((row.start - upstream <= signal_data.end) & (signal_data.end <= row.end))).mean().fillna(0)[mid])
    methy_downstream.append(signal_data.where(((row.start <= signal_data.start) & (signal_data.start <= row.end + downstream)) | ((row.start <= signal_data.end) & (signal_data.end <= row.end + downstream))).mean().fillna(0)[mid])


In [10]:
pickle.dump( in_gene, open( "in_gene_tumor_buckets.p", "wb" ) )

pickle.dump( methy_upstream, open( "methy_upstream_tumor_buckets.p", "wb" ) )

pickle.dump( methy_downstream, open( "methy_downstream_tumor_buckets.p", "wb" ) )

In [12]:
len(in_gene)

28589

In [ ]:
os.getcwd()

In [8]:
in_gene = pickle.load( open( "in_gene.p", "rb" ) )
methy_upstream = pickle.load( open( "methy_upstream.p", "rb" ) )
methy_downstream = pickle.load( open( "methy_downstream.p", "rb" ) )

In [22]:
in_gene = pickle.load( open( "in_gene_tumor_buckets.p", "rb" ) )
methy_upstream = pickle.load( open( "methy_upstream_tumor_buckets.p", "rb" ) )
methy_downstream = pickle.load( open( "methy_downstream_tumor_buckets.p", "rb" ) )

In [23]:
col_names = []
for i in range(20):
    col_names.append("in_gene {}".format(i))
in_gene_df = pd.DataFrame(in_gene, columns = col_names)

col_names = []
for i in range(10):
    col_names.append("upstream {}".format(i))
upstream_df = pd.DataFrame(methy_upstream, columns = col_names)

col_names = []
for i in range(10):
    col_names.append("downstream {}".format(i))
downstream_df = pd.DataFrame(methy_downstream, columns = col_names)

In [24]:
X = pd.concat([in_gene_df,upstream_df,downstream_df], axis=1)
y = pd.Series(dataF["lung___tumor"])

In [19]:
X = pd.DataFrame({'at_gene': in_gene, 'upstream_1000': methy_upstream, 'downstream_3000': methy_downstream}) 
y = pd.Series(dataF["lung___tumor"])

X ["expr"] = y
X = X.dropna()
y = X ["expr"]
X = X[["at_gene", "upstream_1000", "downstream_3000"]]

In [25]:
X

,in_gene 0,in_gene 1,in_gene 2,in_gene 3,in_gene 4,in_gene 5,in_gene 6,in_gene 7,in_gene 8,in_gene 9,...,downstream 0,downstream 1,downstream 2,downstream 3,downstream 4,downstream 5,downstream 6,downstream 7,downstream 8,downstream 9
0,0.001413,0.000705,0.000001,0.002982,0.001681,0.000237,0.001888,0.000000,0.000891,0.00014,...,0.001171,0.001576,0.000980,0.001062,0.000539,0.000718,0.0,0.001323,0.000517,0.000109
1,0.000530,0.000022,0.000000,0.000469,0.000000,0.000083,0.000340,0.000000,0.000154,0.00000,...,0.000333,0.000429,0.000080,0.000189,0.000077,0.000000,0.0,0.000341,0.000199,0.000068
2,0.000530,0.000022,0.000000,0.000469,0.000000,0.000083,0.000340,0.000000,0.000154,0.00000,...,0.000333,0.000429,0.000080,0.000189,0.000077,0.000000,0.0,0.000341,0.000199,0.000068
3,0.000320,0.000264,0.000000,0.000843,0.000039,0.000000,0.000655,0.000000,0.000000,0.00000,...,0.000400,0.000437,0.000124,0.000329,0.000018,0.000072,0.0,0.000501,0.000239,0.000000
4,0.000225,0.000191,0.000000,0.000031,0.000247,0.000003,0.000159,0.000000,0.000405,0.00000,...,0.000301,0.000016,0.000144,0.000177,0.000220,0.000152,0.0,0.000352,0.000151,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28584,0.000000,0.000523,0.000035,0.000000,0.000000,0.000000,0.000036,0.000039,0.000000,0.00000,...,0.000324,0.000018,0.000000,0.000083,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
28585,0.000000,0.000162,0.000000,0.000000,0.000104,0.000000,0.000030,0.000247,0.000000,0.00000,...,0.000081,0.000000,0.000072,0.000172,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
28586,0.000000,0.000162,0.000000,0.000000,0.000104,0.000000,0.000030,0.000247,0.000000,0.00000,...,0.000081,0.000000,0.000072,0.000172,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
28587,0.000000,0.000162,0.000000,0.000000,0.000104,0.000000,0.000030,0.000247,0.000000,0.00000,...,0.000081,0.000000,0.000072,0.000172,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [222]:
from sklearn.model_selection import train_test_split

In [259]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)
n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[0]

In [260]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.core import Reshape

In [261]:
verbose, epochs, batch_size = 0, 50, 85

model = Sequential()
model.add(Conv1D(filters=1, kernel_size=10, activation='relu', input_shape=(n_features, 1)))
# model.add(Dropout(0.5)
# model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(250, activation='relu'))
# model.add(Dense(125, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])
print(model.summary())

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_40 (Conv1D)           (None, 31, 1)             11        
_________________________________________________________________
flatten_40 (Flatten)         (None, 31)                0         
_________________________________________________________________
dense_122 (Dense)            (None, 500)               16000     
_________________________________________________________________
dense_123 (Dense)            (None, 250)               125250    
_________________________________________________________________
dense_124 (Dense)            (None, 1)                 251       
Total params: 141,512
Trainable params: 141,512
Non-trainable params: 0
_________________________________________________________________
None


In [262]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
y_train = np.array(y_train)
# X_train = np.reshape(X_train,(1, X_train.shape[0], X_train.shape[1]))

In [263]:
X_train.shape

(1415, 40, 1)

In [264]:

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
accuracy

0.10734463483095169

In [185]:
accuracy

0.10734463483095169

In [ ]:
sns.pairplot(X)

In [25]:
np.count_nonzero(y) 

1542

In [26]:

# fake_data = np.random.rand(50000,4)
# fake_df = pd.DataFrame({'signal_in_exp': fake_data[:, 0], 'upstream 1000': fake_data[:, 1], 'downstream 3000': fake_data[:, 2], 'expression': fake_data[:, 3]})
model = LinearRegression().fit(X_train, y_train)
print(model.score(X_test, y_test))


0.0623584886969597


In [32]:
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [33]:
svr_poly = SVR(kernel='poly',gamma ='auto')
svr_poly.fit(X_train, y_train)
pred = svr_poly.predict(X_test)
r2_score(y_test, pred)
# svr_poly.score(X_test, y_test)

-0.21593370258836675

In [30]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

-0.0033732316767927095